In [1]:
import argparse
import os
import ray


from typing import *
from ray import air
from ray import tune
from baselines.train.configs import get_experiment_config
from ray.rllib.algorithms import ppo
from ray.tune import registry
from ray.air.integrations.wandb import WandbLoggerCallback
from baselines.train import make_envs

2024-05-23 08:18:50,362	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-23 08:18:51,180	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-23 08:18:52.326970: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-23 08:18:53.453871: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-05-23 08:18:53.453935: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvin

In [2]:
ckp = "results/torch/pd_matrix/PPO_meltingpot_397b4_00000_0_2024-05-21_13-58-39/checkpoint_007270"

registry.register_env("meltingpot", make_envs.env_creator)


2024-05-23 08:19:02,066	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-23 08:19:02,067	WARNING algorithm_config.py:2548 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.
2024-05-23 08:19:02,069	WARNING algorithm_config.py:656 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2024-05-23 08:19:02,074	WARNIN

Continuing training from results/torch/pd_matrix/PPO_meltingpot_397b4_00000_0_2024-05-21_13-58-39/checkpoint_007270


In [6]:

trainer:ppo.PPO = ppo.PPO.from_checkpoint(ckp)
trainer.restore(ckp)
print(f"Continuing training from {ckp}")

2024-05-23 08:22:08,602	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-23 08:22:08,603	WARNING algorithm_config.py:2548 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.
2024-05-23 08:22:08,609	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploratio

Continuing training from results/torch/pd_matrix/PPO_meltingpot_397b4_00000_0_2024-05-21_13-58-39/checkpoint_007270


In [8]:
print(trainer)
print(dir(trainer))
print(trainer.__dict__)

PPO
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__ray_actor_class__', '__ray_ready__', '__ray_terminate__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_allow_unknown_configs', '_allow_unknown_subkeys', '_annotated', '_automatic_evaluation_duration_fn', '_before_evaluate', '_checkpoint_cls', '_checkpoint_info_to_algorithm_state', '_close_logfiles', '_compile_iteration_results', '_counters', '_create_checkpoint_dir', '_create_local_replay_buffer_if_necessary', '_create_logger', '_env_id', '_episode_history', '_episodes_to_be_collected', '_episodes_total', '_evaluate_async', '_export_model', '_get_env_id_and_creator', '_get_latest_available_checkpoint', '_get_latest_local_available_c